In [33]:
import re
import requests
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os, glob
import time
import pandas as pd
import numpy as np
    
# Tole dvoje je treba, da se Chrome sproti ne odpira
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)

C:\Users\jasak\AppData\Local\Temp\ipykernel_20420\159072978.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=chrome_options)


In [34]:
vzorec_bloka = re.compile(
    r'<td><mtg-set-icon.*?'
    r'</tr',
    flags=re.DOTALL
)

vzorec_karte = re.compile(
    r'<td><mtg-set-icon.*?>.*?<i _ngcontent.*? class="ss ss-fw ss-(?P<set>.+?)\b( ss-.*?)?"></i><!----></mtg-set-icon>'
    r'<a href="/prints/(?P<id_karte>\d*).*?">'
    r'(?P<ime>.*?)'
    r'</a>.*?</td>.*?<td> (?P<redkost>.*?) </td>'
    r'.*?<td class="text-end">'
    r' (?P<povprecna_cena>.*?) </td>(<\!---->)+(<td class="text-end">.*?</td>.*?<td class="text-end"> (?P<povprecna_cena_foil>.*?) </td>.*?<td class="text-end">.*?</td>)?',  
    flags=re.DOTALL
    )

def izloci_podatke_o_kartah(blok):
    karta = vzorec_karte.search(blok).groupdict()
    karta['id_karte'] = int(karta['id_karte'])
    karta['ime'] = karta['ime'].strip()
    karta['set'] = karta['set'].upper()
    karta['redkost'] = karta["redkost"].strip()
    karta['povprecna_cena'] = karta["povprecna_cena"]
    karta['povprecna_cena_foil'] = karta["povprecna_cena_foil"]

    return karta

def izloci_podatke_o_specificni_karti(vsebina):
    """
    To bo izločilo vse dodatne podatke o karti, ki jih bomo našli na naslovu .../prints/<id_karte>
    """
    pass

def pridobi_ustrezno_ime_lokalne_datoteke(st_strani):
    return f"Podatki o kartah/Podatki iz setov/Karte iz seta st. {st_strani}.html"

In [35]:
# Najprej poradiramo csv in json datoteki
with open("karte.csv", "w", encoding = "utf-8") as dat:
    pass

with open("karte.json", "w", encoding = "utf-8") as dat:
    pass

PRVI_SET = 1
ZADNJI_SET = 1300

#indeksi_setov_ki_obstajajo = []

for st_strani in range(PRVI_SET, ZADNJI_SET + 1):
    url = (
        f'https://www.mtgstocks.com/sets/{st_strani}' 
    )
    print(f"\n Zajemam {url}\n")
    driver.get(url)
    time.sleep(1)
    
    
    vsebina = driver.page_source
    naslov_strani = driver.title
    
    stevilo_podstrani_s_kartami = vsebina.count("pagination-page page-item")
    st_besed_Market_na_strani = vsebina.count(">Market<")
    
    print("Naslov strani:", naslov_strani)
    
    if naslov_strani in ["File not found - MTGStocks"]:
        print(f'\n Stran št. {st_strani} ni dosegljiva.\n')
    else:
        print(f'\n Stran št. {st_strani} JE dosegljiva.\n')
        if "Art Series:" in naslov_strani or "From the Vault:" in naslov_strani or st_besed_Market_na_strani == 0:
            continue
        else:
            with open(pridobi_ustrezno_ime_lokalne_datoteke(st_strani), 'w', encoding = "utf-8") as dat:
                driver.find_element(by=By.XPATH, value = "//*[text() = 'Market']//..").click()
                for podstran in range(1, stevilo_podstrani_s_kartami + 1):
                    driver.find_element(by=By.XPATH, value = f"//*[contains(@class, 'pagination-page page-item')][{podstran}]//a").click()
                    time.sleep(1)
                    print(f"Pobrana {podstran}. podstran")
                    dat.write(driver.page_source)
  


 Zajemam https://www.mtgstocks.com/sets/1

Naslov strani: Return to Ravnica - MTGStocks

 Stran št. 1 JE dosegljiva.

Pobrana 1. podstran


KeyboardInterrupt: 

Koda za zapisovanje kart iz lokalnih datotek v csv in json format.


In [55]:
debug_mode_za_branje_iz_lokalnih_datotek = False
path = 'Podatki o kartah/Podatki iz setov'
karte = []

for filename in glob.glob(os.path.join(path, '*.html')):
    with open(filename, 'r', encoding = "utf-8") as f:

        stetje = 0
        print(f"Berem podatke kart iz {filename}")
        vsebina = f.read()
        
        for blok in vzorec_bloka.finditer(vsebina):
            stetje += 1
            
            if debug_mode_za_branje_iz_lokalnih_datotek:
                print(stetje)
                print(blok.group(0))
                print(vzorec_karte.search(blok.group(0)).groupdict())
            
            karte.append(izloci_podatke_o_kartah(blok.group(0)))

with open("karte.json", "a", encoding = "utf-8") as dat:
    json.dump(karte, dat, indent=4, ensure_ascii=False)
    

with open("karte.csv", "a", encoding = "utf-8") as dat:
    writer = csv.DictWriter(dat, [
        "id_karte",
        "ime",
        "set",
        "redkost",
        "povprecna_cena",
        "povprecna_cena_foil",
    ])
    writer.writeheader()
    writer.writerows(karte)

Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 1.html
Berem podatke kart iz Podatki o kartah/Podatki iz setov\Karte iz seta st. 10.html


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 99824: invalid start byte

Zdaj se znebimo kart, ki so tokeni ali basic landi.
Prav tako bi se radi znebili kart, ki imajo tako ceno kot ceno foila ničelno. (Torej da lahko karte primerjamo po vsaj eni komponenti).
Pri tem je vredno omeniti, da imajo karte, ki so najbolj zaželene, tudi najbolj popolne podatke, torej jih s tem ne bomo izgubili.


In [60]:
podatki_o_kartah = pd.read_csv("karte.csv", encoding="latin1")

regex_izraz_za_basic_lande = r'Forest \(\d|Mountain \(\d|Swamp \(\d|Plains \(\d|Island \(\d|Wastes \(\d'
okrajsani_podatki = podatki_o_kartah[~(podatki_o_kartah.redkost.isin(["Token", "Land"])) & ~(podatki_o_kartah.ime.str.match(regex_izraz_za_basic_lande)) & ~(podatki_o_kartah.ime.isin(["Forest", "Mountain", "Swamp", "Plains", "Island", "Wastes"]))]
okrajsani_podatki = okrajsani_podatki[~(okrajsani_podatki.povprecna_cena.isnull() & okrajsani_podatki.povprecna_cena_foil.isnull())]
okrajsani_indeksi = okrajsani_podatki.id_karte
print(okrajsani_podatki)
print(okrajsani_indeksi)

       id_karte               ime  set   redkost povprecna_cena  \
0         19748     Cyclonic Rift  RTR      Rare         $38.00   
1         19934   Utvara Hellkite  RTR    Mythic         $19.53   
2           360       Blood Crypt  RTR      Rare         $19.99   
3           404       Steam Vents  RTR      Rare         $18.63   
4           395    Overgrown Tomb  RTR      Rare         $13.99   
...         ...               ...  ...       ...            ...   
47532     18337  Dregscape Zombie  ARC    Common          $0.25   
47533     18380        Juggernaut  ARC  Uncommon          $0.25   
47534     18421           Plummet  ARC    Common          $0.25   
47535     18348          Fertilid  ARC    Common          $0.38   
47536     18434  Sanctum Gargoyle  ARC    Common          $0.24   

      povprecna_cena_foil  
0                  $57.99  
1                  $57.44  
2                  $34.37  
3                  $38.61  
4                  $25.72  
...                   ...  

Zdaj pa pogledamo vsako posamezno karto. Zato bomo naredili posamezne vzorce, ki nam bodo pomagali pri parsanju.
Nesmiselno je imeti samo en vzorec, saj je to samo logistična nočna mora.


In [63]:
# vzorec_specificne_karte = re.compile(
#     r'Europe avg </p> (?P<povprecje_eu>.*?) </mtg.*class="card-oracle-item".*?</i> (?P<oracle_text>.*?) <.* '
#     r'All Time Low </p><h3.*?>(?P<all_time_low>.*?)</h3><h5.*?> (?P<datum_all_time_low>.*?) </h5>.*?'
#     r'All Time High </p><h3.*?>(?P<all_time_high>.*?)</h3><h5.*?> (?P<datum_all_time_high>.*?) </h5>.*?'
#     r'Reserved list\?</p><h3.*?>(?P<reserved_list>.*?)</h3>.*?'
#     ,
#     flags=re.DOTALL
# )

vzorec_povprecja_eu = re.compile(
    r'Europe avg </p> (?P<povprecje_eu>.*?) <',
    flags=re.DOTALL
)

vzorec_oracle_texta = re.compile(
    r'class="card-oracle-item".*?</i> (?P<oracle_text>.*?) <',
    flags=re.DOTALL
)

vzorec_all_time_low = re.compile(
    r'All Time Low </p><h3.*?>(?P<all_time_low>.*?)</h3><h5.*?> (?P<datum_all_time_low>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_all_time_high = re.compile(
    r'All Time High </p><h3.*?>(?P<all_time_high>.*?)</h3><h5.*?> (?P<datum_all_time_high>.*?) </h5>',
    flags=re.DOTALL
)

vzorec_reserved_list = re.compile(
    r'Reserved list\?</p><h3.*?>(?P<reserved_list>.*?)</h3>',
    flags=re.DOTALL
)

In [94]:
with open("testni_zajcek.txt", "r", encoding = "utf-8") as testna_datoteka:
    vsebina = testna_datoteka.read()
    
def izlusci_podatke_iz_glavnine(niz):
    povprecje_eu = re.findall(vzorec_povprecja_eu, niz)
    oracle_text = re.findall(vzorec_oracle_texta, niz)
    reserved_list = re.findall(vzorec_reserved_list, niz)
    
    if len(povprecje_eu) == 1:
        povprecje_eu = povprecje_eu[0]
    else:
        povprecje_eu = None
    if len(oracle_text) == 1:
        oracle_text = oracle_text[0]
    else:
        oracle_text = None
    if len(reserved_list) == 1:
        reserved_list = reserved_list[0]
    else:
        reserved_list = None
    
    najdi_vse_low = re.findall(vzorec_all_time_low, niz)
    najdi_vse_high = re.findall(vzorec_all_time_high, niz)
    if len(najdi_vse_low) == 1:
        if len(najdi_vse_low[0]) == 2:
            all_time_low, all_time_low_datum = najdi_vse_low[0][0], najdi_vse_low[0][1]
    else:
        all_time_low, all_time_low_datum = None, None
    if len(najdi_vse_high) == 1:
        if len(najdi_vse_high[0]) == 2:
            all_time_high, all_time_high_datum = najdi_vse_high[0][0], najdi_vse_high[0][1]
    else:
        all_time_high, all_time_high_datum = None, None
    return {"povprecje_eu": povprecje_eu, "oracle_text": oracle_text, "reserved_list":reserved_list, "all_time_low": all_time_low, "all_time_high": all_time_high}

izlusci_podatke_iz_glavnine(vsebina)

{'povprecje_eu': None,
 'oracle_text': None,
 'reserved_list': None,
 'all_time_low': None,
 'all_time_high': None}

In [58]:
print("Število vseh indeksov kart, ki jih imamo:", len(okrajsani_indeksi))
for indeks_printa in okrajsani_indeksi[:300]:
    url = f"https://www.mtgstocks.com/prints/{indeks_printa}"
    print(f"Zajemanje printa karte št. {indeks_printa} s strani {url}")
    with open(f"Podatki o kartah/Podatki o specificnih kartah/Podatki o karti st. {indeks_printa}.html", "w", encoding="utf-8") as dat:
        try:
            driver.get(url)
            time.sleep(1)
            dat.write(driver.page_source)
        except Exception as e:
            print(f"Prišlo je do napake pri setu {indeks_printa}")
            print(e)
        

Število vseh indeksov kart, ki jih imamo: 43057
Zajemanje printa karte št. 19748 s strani https://www.mtgstocks.com/prints/19748


C:\Users\jasak\AppData\Local\Temp\ipykernel_20420\3063670685.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for indeks_printa in okrajsani_indeksi[:300]:


Zajemanje printa karte št. 19934 s strani https://www.mtgstocks.com/prints/19934


KeyboardInterrupt: 

In [50]:
vzorec_za_manacost = r'ms ms-cost ms-(?P<mana_cost>.*?)["| ]'

def preberi_manacost_in_barvo(niz):
    seznam_simbolov = re.findall(vzorec_za_manacost, niz)
    cmc = 0
    manacost = ""
    barva = ""
    for simbol in seznam_simbolov:
        manacost += simbol
        if simbol.isnumeric():
            cmc += int(simbol)
        elif simbol in "wubrg":
            cmc += 1
            if simbol not in barva:
                barva += simbol
        else:
            if simbol not in "{(p)}":
                cmc += 1
            if simbol not in "{(p)}c":
                print("Program je našel nenavaden simbol", simbol)
    
    if barva == "":
        barva += "c"
    if manacost == "":
        manacost += "0"
    return {"cmc": cmc, "manacost": manacost, "barva": barva}
    
string_za_poskus = ''
print(re.findall(vzorec_za_manacost, string_za_poskus))

print(preberi_manacost_in_barvo(string_za_poskus))

[]
{'cmc': 0, 'manacost': '0', 'barva': 'c'}
